# Assembling your MLP

As mentioned in the previous challenge, the missing brick for you to be able to update your MLP parameters during training is to backpropagate our update computation.

In [1]:
import numpy as np
np.random.seed(42)

## Loss minimization

Using the previous matrix understanding of a layer forward operation:
$$ \forall\ W\ \in\ \mathbb{M}_{M,N}(\mathbb{R}), \forall\ b\ \in\ \mathbb{R}^M, \forall\ X\ \in\ \mathbb{R}^N,\ f(X) = activation(WX + b) $$

Assuming that we expected the output to be $y \in \mathbb{R}$, we can define a loss function, Mean Squared Errors (MSE) in this case:

$$ \forall y \in \mathbb{R}^M, \forall X \in \mathbb{R}^N, L(X, y) = \frac{1}{M} \sum_{i=1}^M (f(X)_i - y_i)^2
= \frac{1}{M} (f(X) - y)^T * (f(X) - y)
$$

Further resources on loss functions:
- Loss functions for different problems: [Christopher Bourez's blog](http://christopher5106.github.io/deep/learning/2016/09/16/about-loss-functions-multinomial-logistic-logarithm-cross-entropy-square-errors-euclidian-absolute-frobenius-hinge.html)
- Debunking loss functions in deep learning: shorter but better illustrated [Medium article](https://medium.com/@dmitrijtichonov/debunking-loss-functions-in-deep-learning-4b9abc4c8d4c) by Dmitrij Tichonov

Since we will be optimizing the parameters of our MLP relatively to the loss, we will need a function to compute it. 

In [2]:
def compute_loss(expected, output):
    """
    Compute the loss for a given expected and computed output
    Args:
        expected (np.array): 1D-numpy array
        output (np.array): 1D-numpy array of same size as expected
    Returns:
        loss (float): loss respective to expected and output
    """
    
    # TODO: compute the MSE loss
    squared_errors = None
    
    return None

Check that the returned object is a scalar, and since we initialize the inputs between 0 and 1, the loss should be as well.

In [3]:
output_size = 8
# Random values for loss computation
output_values = np.random.rand(output_size)
expected_values = np.random.rand(output_size)

print(compute_loss(expected_values, output_values))

0.21204441770908766


Our objective is to minimize the Loss to be as close as possible to y, and in order to find a minimum, what better option is there than derivation.

As we aim at finding minima of the loss function (where the gradient of the loss function is zero), we will update our weights as follows:
$$ W \leftarrow W - \alpha \frac{\partial L(X,y)}{\partial W} = W - \alpha \nabla_L(X,y) $$
where $\alpha$ is called the learning rate and $\nabla_L \in \mathbb{M}_{M,N}(\mathbb{R})$ represents the gradient of L relatively to the weights.

![Gradient descent](https://cdn-images-1.medium.com/max/800/1*HrFZV7pKPcc5dzLaWvngtQ.png "Gradient descent on loss")
*Source: http://sebastianraschka.com/ — Python Machine Learning, 2nd Edition*

More specifically, the gradient is a differential operator for multi-variate functions. This specific optimizer is a basic Gradient Descent. There are many other optimizers available, check the following resources:
- List of illustrated optimizers: [Medium article](https://towardsdatascience.com/types-of-optimization-algorithms-used-in-neural-networks-and-ways-to-optimize-gradient-95ae5d39529f) by Anish Singh Walia, and [blog post](http://ruder.io/optimizing-gradient-descent/) by Sebastian Ruder
- Deep Learning book by Ian Goodfellow: [Optimization for training Deep Learning](https://www.deeplearningbook.org/contents/optimization.html)
- Highlights of 2017 for deep learning optimization: [recent updates](http://ruder.io/deep-learning-optimization-2017/)
- 3Blue1Brown videos: [Gradient Descent](https://www.youtube.com/watch?v=IHZwWFHWa-w&t=1s)

## A single perceptron update

Let's take a step back to our single perceptron model with a forward operation $f: \mathbb{R}^N \rightarrow \mathbb{R}$
$$ \forall X \in\ \mathbb{R}^N,\ f(X) = activation\Bigg(\Big(\sum_{i=1}^{N} W_i * X_i\Big) + b\Bigg) $$

As we have a single perceptron, there is only one term in the sum of the loss function:

$$ \forall y \in \mathbb{R}, \forall X \in \mathbb{R}^N, L(X, y) = (f(X) - y)^2 $$

In order to compute the gradient, we are going to need partial derivatives of f. But since it's a composed function, the chain rule is going to be useful.

### Chain rule
In this challenge, we will have to compute derivatives of complex functions. One can compute the derivative of a composed functions using the chain rule. If we define the function f as being the composition of g by h, on a given space called E:
$$ \forall x \in E,\ f(x) = h(g(x)) $$
then
$$ \forall x \in E, f'(x) = \frac{df}{dx}(x) = \frac{dh}{dg(x)}(g(x)) * \frac{dg}{dx}(x) = h'(g(x)) * g'(x) $$

Time to get to work, let's find the gradient of our loss function!

$$ \forall y \in \mathbb{R}, \forall X \in \mathbb{R}^N,\ 
\forall i \in [1, N],$$
$$
\frac{\partial L(X, y)}{\partial W_i} = \frac{\partial}{\partial W_i}(f(X) - y)^2
= 2(f(X) - y)\frac{\partial f(X)}{\partial W_i}
$$

Interesting, we now have to calculate the partial derivative of the forward function:

$$ \forall X \in\ \mathbb{R}^N,\ \forall i \in [1, N],$$
$$
\frac{\partial f(X)}{\partial W_i} = 
activation'\Bigg(\Big(\sum_{j=1}^{N} W_j * X_j\Big) + b\Bigg) * 
\frac{\partial}{\partial W_i}\Bigg(\Big(\sum_{j=1}^{N} W_j * X_j\Big) + b\Bigg) = activation'\Bigg(\Big(\sum_{j=1}^{N} W_j * X_j\Big) + b\Bigg) * X_i $$

So, in a nutshell,
$$ \forall y \in \mathbb{R}, \forall X \in \mathbb{R}^N,\ \forall i \in [1, N],$$
$$
\frac{\partial L(X, y)}{\partial W_i} = 2(f(X) - y) * activation'\Bigg(\Big(\sum_{j=1}^{N} W_j * X_j\Big) + b\Bigg)*X_i
$$
$$
\frac{\partial L(X, y)}{\partial b} = 2(f(X) - y) * activation'\Bigg(\Big(\sum_{j=1}^{N} W_j * X_j\Big) + b\Bigg)
$$

- as we have already computed the weighted sum with the bias on the forward pass, the only unknown here is the derivative of the activation function
- since the activation is predefined when we create our MLP, we can express analytically its derivative and predefined it just like the activation.

### An activation example
First let's take back the sigmoid $\sigma$ as an activation function and it's derivative $\sigma': \mathbb{R} \rightarrow \mathbb{R}$, 

$$ \forall\ x \in \mathbb{R},\ \sigma'(x) = \frac{d\sigma}{dx}(x) = \frac{d}{dx}\Bigg(\frac{1}{1 + e^{-x}}\Bigg) = \frac{e^{-x}}{(1 + e^{-x})^2} = \sigma(x) \frac{e^{-x}}{1 + e^{-x}} = \sigma(x) \frac{(1 + e^{-x}) - 1}{1 + e^{-x}} $$
$$ \forall\ x \in \mathbb{R},\ \sigma'(x) = \frac{d\sigma}{dx}(x) = \sigma(x) (1 - \sigma(x)) $$

In [4]:
## TODO: define right here the expressions of sigmoid and sigmoid_prime 
def sigmoid(x): return 0
def sigmoid_prime(x): return 0

Using what we said about predefining both the activation and its derivative, edit the Activation class constructor and methods of the perceptron.py file. Let the inputs (0) as is, since it's easier to check the value of the sigmoid in zero than other values.

In [5]:
## TODO: edit the Activation methods __init__, forward and prime
from perceptron import Activation
activation = Activation(sigmoid, sigmoid_prime)
print(activation.forward(0), activation.prime(0))

0.5 0.25


Now that we have a more versatile object for activation, we need to work on the weight update.
During the forward pass, as we compute the weighted sum with the bias, we will store the value of $\sigma'$ for this weighted sum. Edit the Perceptron class constructor and methods of the perceptron.py file by following the instructions in comments

In [6]:
input_size = 16
## TODO: edit the Perceptron methods __init__, forward, backward, update_params()
from perceptron import Perceptron
p = Perceptron(input_size, activation=activation, requires_grad=True)

Let's test your implementation by performing a small training sequence. For further tests, we will use the [fastprogress](https://github.com/fastai/fastprogress) library to track our progress during training. In this notebook, we will not consider monitoring the validation loss or concern about overfitting, but only about the convergence of our training loss to a minima.

In [7]:
from fastprogress import master_bar, progress_bar
# Hyperparameters (feel free to change them)
alpha = 1e-3
nb_iterations = 1000
nb_epochs = 10

# Inputs & expected outputs
inputs = np.random.rand(nb_iterations, input_size)
# Let's try to approximate the normalized sum (mean) of the input
# Note: we can't just sum as the perceptron output is squeezed by our activation choice
# Note: feel free to change the activation to ReLU for instance
y = inputs.sum(axis=1) / inputs.shape[1]
# Training Loop
mb = master_bar(range(nb_epochs))
for epoch in mb:
    loss = 0
    for iter_idx in progress_bar(range(nb_iterations), parent=mb):
        # TODO: forward the input
        output = None
        # TODO: backpropagate the gradient of the loss
        p.backward(None)
        # TODO: update the weights and bias of the perceptron
        p.update_params(0)
        # TODO: update the total loss (do not divide by the number of terms)
        loss += 0
        mb.child.comment = f"running loss {loss / float(iter_idx + 1)}"
    mb.first_bar.comment = f"completed epoch {epoch+1}"
    mb.write(f"Epoch {epoch+1}: MSE {loss / float(nb_iterations)}")

You managed to run your first training for your own perceptron, congratulations! Perhaps you feel like we should upgrade to something closer to an MLP? Well, that's exactly what we are going to do.

## MLP backpropagation

Let an MLP of K layers with their respective:
- width $(s_l)_{l \in [1,K]} \in \mathbb{N}^K$ (we'll note $s_0$ the number of inputs of the first layer), 
- weights $(W_l)_{l \in [1,K]} = ((w_{l,i,j})_{(i,j) \in [1, s_l] \times [1, s_{l-1}]})_{l \in [1,K]}$,
- biases $(B_l)_{l \in [1,K]} = ((b_{l,i})_{i \in [1, s_l]})_{l \in [1,K]}$,
- layer activation values $(A_l)_{l \in [1, K]} = ((a_{l,i})_{i \in [1, s_l]})_{l \in [1,K]}$,
- weighted sum with bias $ (Z_l)_{l \in [1,K]} = ((z_{l,i})_{i \in [1, s_l]})_{l \in [1,K]} = (W_l A_{l-1} + B_l)_{l \in [1,K]}$

![MLP](http://pubs.sciepub.com/ajmm/3/3/1/bigimage/fig5.png "Multi-layer perceptron")
*Source: Science and Education Publishing*

So more generally, we can define our update rule for every weight matrix in our MLP.
$$
\forall l \in [1, K],\
W_l \leftarrow W_l - \alpha \frac{\partial L(X,y)}{\partial W_l} = W_l - \alpha \nabla_L^l(X, y) $$
where $\nabla_L^l$ represents the gradient of the loss L relatively to the weights of layer l.

$$
\forall l \in [1, K],\
\forall y \in \mathbb{R}^{S_K}, \forall X \in \mathbb{R}^{S_0},\ $$
$$
\nabla_L^l(X, y) = \begin{bmatrix}
    \frac{\partial L(X,y)}{\partial w_{l,1,1}} & \cdots & \frac{\partial L(X,y)}{\partial w_{l,1,s_{l-1}}} \\
    \vdots & & \vdots \\
    \frac{\partial L(X,y)}{\partial w_{l,s_l,1}} & \cdots & \frac{\partial L(X,y)}{\partial w_{l,s_l,s_{l-1}}}
\end{bmatrix} = \Big(\frac{\partial L(X,y)}{\partial w_{l,i,j}}\Big)_{(i,j) \in [1,s_{l-1}] \times [1,s_l]} $$

Here is a list of resources you might want to check for matrix calculus:
- Videos: Ben Lambert [Part 1](https://www.youtube.com/watch?v=iWxY7VdcSH8) [Part 2](https://www.youtube.com/watch?v=uoejt0FCWWA) [Part 3](https://www.youtube.com/watch?v=i6fqfH5hx60)
- Lecture: National University of Singapore [Matrix Differentiation](https://www.comp.nus.edu.sg/~cs5240/lecture/matrix-differentiation.pdf)
- Notes: University of Minnesota [Matrix differentiation](https://atmos.washington.edu/~dennis/MatrixCalculus.pdf)

![Backpropagation](https://cdn-images-1.medium.com/max/1600/1*FceBJSJ7j8jHjb4TmLV0Ew.png "Backpropagation illustration")

*Source: Stanford CS231n - Lecture 4*

Using our first definition of the loss function with our updated notation,
$$
\forall X \in \mathbb{R}^{s_0}, \forall y \in \mathbb{R}^{s_K},
L(X, y) = \frac{1}{s_K} \sum_{j=0}^{s_K} (a_{K,j} - y_j)^2
$$

Fortunately, with the chain rule, we can express this loss gradient in a more convenient way:

$$ 
\forall X \in \mathbb{R}^{s_0}, \forall y \in \mathbb{R}^{s_K}, \forall l \in [1, K],\
$$
$$
\frac{\partial L(X, y)}{\partial W_l} = \frac{\partial L(X, y)}{\partial A_K} \frac{\partial A_K}{\partial W_l}
= \frac{1}{s_K} \Bigg(\frac{\partial}{\partial a_{K,i}} \Bigg(\sum_{j=1}^{s_K} (a_{K,j}(X) - y_j)^2 \Bigg)\Bigg)_{i \in [1, s_K]} \frac{\partial A_K}{\partial W_l}
$$
$$
= \frac{1}{s_K} \Bigg(\frac{\partial}{\partial a_{K,i}} \Bigg((a_{K,i}(X) - y_i)^2 \Bigg)\Bigg)_{i \in [1, s_K]} \frac{\partial A_K}{\partial W_l}
$$

Thus by introducing the error $ \mathcal{E}(X,y) = (\epsilon_i(X,y_i))_{i \in [1, s_K]} = (a_{K,i}(X) - y_i)_{i \in [1, s_K]}$,

$$ 
\forall X \in \mathbb{R}^{s_0}, \forall y \in \mathbb{R}^{s_K}, \forall l \in [1, K],\
$$
$$
\frac{\partial L(X, y)}{\partial W_l} = \frac{2}{s_K} \Big(\epsilon_i(X,y_i)\Big)_{i \in [1, s_K]} \frac{\partial A_K}{\partial W_l}
= \frac{2}{s_K} \mathcal{E}(X,y) \frac{\partial A_K}{\partial W_l}
$$

The last part of the expression shows that the gradient of the loss function is directly related to the gradient of the forward function. This will allow us to update the last layer parameters but we will to backpropagate the gradient further to update earlier layers.

### Computational Graph

As we will need to recurrently calculate the gradient relatively to the weights and biases for updating our MLP parameters, the notion of a computational graph is extremely useful.

![Computational graph](https://cdn-images-1.medium.com/max/1400/1*UATNEUQ0dKbvDdI79y3z9A.png "Computational graph")

*Source: Manik Sonik - Hackernoon: Exploding And Vanishing Gradient Problem: Math Behind The Truth*

The core idea is to decompose the function performed by the entire MLP into elementary operations *(cf. the 4-layer MLP above)*. In our case, the operations are:
- Matrix by vector multiplication (on weights and inputs)
- Vectors' sum (on the weighted sum and the bias)
- Activation (on the $(Z_l)_{l \in [1, K]}$)

Iteratively you can map the entire MLP with these operations, that are individually easy to calculate the gradient of. Here is an excellent resource about backpropagation:

- 3Blue1Brown videos: [Introducation to backpropagation](https://www.youtube.com/watch?v=Ilg3gGewQ5U) [Backpropagation calculus](https://www.youtube.com/watch?v=tIeHLnjs5U8)

### Propagating gradients through the computational graph

Using the chain rule, we are going to need the relation between the partial derivative of the loss function relatively to a given layer's parameters ($W_l, B_l$) & inputs ($A_{l-1}$), and the one relatively to the previous layer's parameters & inputs.

First, we need to derive through the activation gate:
$$
\forall X \in \mathbb{R}^{s_0}, \forall y \in \mathbb{R}^{s_K}, \forall l \in [1, K],
\frac{\partial L(X,y)}{\partial Z_l} = \frac{\partial L(X,y)}{\partial A_l} \frac{\partial A_l}{\partial Z_l} = \frac{\partial L(X,y)}{\partial A_l} activation'(Z_l)
$$

We will use the Kronecker symbol $\delta_{ij}$ where $(i,j) \in \mathbb{N}^2$ ,that is equal to 1 if and only if $i=j$. Now using the chain rule, we can use the result to limit computation for the other gradients:

$$
\forall X \in \mathbb{R}^{s_0}, \forall y \in \mathbb{R}^{s_K}, \forall l \in [1, K],
$$
$$
\frac{\partial L(X,y)}{\partial W_l} = \frac{\partial L(X,y)}{\partial Z_l} \frac{\partial Z_l}{\partial W_l}
= \frac{\partial L(X,y)}{\partial Z_l} \Big(\frac{\partial z_{l,i}}{\partial w_{l,j,k}}\Big)_{(i,j,k) \in [1, s_l]^2 \times [1, s_{l-1}]}
= \frac{\partial L(X,y)}{\partial Z_l} \Big(\delta_{ij} a_{l-1,k}\Big)_{(i,j,k) \in [1, s_l]^2 \times [1, s_{l-1}]}
$$
$$
\frac{\partial L(X,y)}{\partial B_l} = \frac{\partial L(X,y)}{\partial Z_l} \frac{\partial Z_l}{\partial B_l}
= \frac{\partial L(X,y)}{\partial Z_l} \Big(\frac{\partial z_{l,i}}{\partial b_{l,j}}\Big)_{(i,j) \in [1, s_l]^2}
= \frac{\partial L(X,y)}{\partial Z_l} (\delta_{ij})_{(i,j) \in [1, s_l]^2}
= \frac{\partial L(X,y)}{\partial Z_l} I_{s_l} = \frac{\partial L(X,y)}{\partial Z_l}
$$
$$
\frac{\partial L(X,y)}{\partial A_{l-1}} = \frac{\partial L(X,y)}{\partial Z_l} \frac{\partial Z_l}{\partial A_{l-1}}
= \frac{\partial L(X,y)}{\partial Z_l} \Big(\frac{\partial z_{l,i}}{\partial a_{l-1,j}}\Big)_{(i,j) \in [1, s_l]^2}
= \frac{\partial L(X,y)}{\partial Z_l} \Big(w_{l,i,j}\Big)_{(i,j) \in [1, s_l]^2}
= \frac{\partial L(X,y)}{\partial Z_l} W_l
$$

Therefore, by recurrence, starting from $A_K$, we multiply the current gradient of the loss by:
- $activation'(Z_l)$ to get the gradient relatively to bias
- $activation'(Z_l) \Big(\delta_{ij} a_{l-1,k}\Big)_{(i,j,k) \in [1, s_l]^2 \times [1, s_{l-1}]}$ to get the gradient relatively to weights
- $activation'(Z_l) W_l$ to get the one for the previous layer

Note that $\Big(\delta_{ij} a_{l-1,k}\Big)_{(i,j,k) \in [1, s_l]^2 \times [1, s_{l-1}]}$ is a 3D-tensor, just like multi-channel image data. But here it would have $s_{l-1}$ channels, and on a given channel $ k \in [1, s_{l-1}]$, it would look like this:

$$
\Big(\delta_{ij} a_{l-1,k}\Big)_{(i,j) \in [1, s_l]^2} =
\begin{bmatrix}
    a_{l-1,k} & & \mathbb{0} \\
     & \ddots & \\
    \mathbb{0} & & a_{l-1,k}
\end{bmatrix}
$$

Keep that in mind when you will compute the loss gradient relatively to the weights *(as it is 3-dimensional, the numpy.matmul will not work, you should check the [numpy.tensordot](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.tensordot.html) function instead)*.

### Layer update
Let's start with a single layer and implement each method. 

Edit the constructor, forward, backward and update_params methods of the Layer class in the mlp.py file (you can copy paste your implementation of the Activation class from perceptron.py). Make sure to read carefully the instructions as you might end up with dimension mismatch at some point!

In [8]:
input_size = 16
output_size = 8
## TODO: edit the Layer methods __init__, forward, backward and update_params
from mlp import Layer
l = Layer(input_size, output_size, activation=activation, requires_grad=True)

This cell will test the dimensions of your network output and of the backpropagated gradient

In [9]:
# Inputs & expected outputs
inputs = np.random.rand(input_size)
# Let's take the sum function as target
expected = inputs.sum() * np.ones(output_size)
# Forward
output = l.forward(inputs)
# Check if you output and backpropagated gradient dimensions are correct
assert output.size == output_size
assert l.backward(2. / output.size * (output - expected)).size == input_size
print("Alright, alright, alright")

Alright, alright, alright


Excellent work, we are almost there!
Now we have to be able to stack layers consecutively and perform the same forward and backward to train our MLP.

### The MLP strikes back

In [10]:
# Choose the number of inputs (feel free to change this)
input_size = 8
output_size = 8
inputs = np.random.rand(input_size)
expected = np.random.rand(output_size)
# We'll define the number of units in each consecutive layer (feel free to change this)
layer_output_sizes = [64, 32, 32, 32, output_size]

This is the part where you just have to use your previous methods of each layer smartly to give a coherence to your MLP. Edit the constructor, forward, backward and update_params methods of the MLP class in the mlp.py file. Again, the instructions will hopefully guide you towards a good implementation.

In [11]:
## TODO: edit the MLP methods __init__ and forward to compute the correct network output
from mlp import MLP
mlp = MLP(input_size, layer_output_sizes, activation=activation, requires_grad=True)

Let's train this network!
With all the work you have done, you will notice that it does not take more python lines to run a training here than with common deep learning frameworks. Just a last effort to get this training started by following the instructions:

In [12]:
from fastprogress import master_bar, progress_bar
# Hyperparameters (feel free to change them)
alpha = 1e-3
nb_iterations = 1000
nb_epochs = 10

# Inputs & expected outputs
inputs = np.random.rand(nb_iterations, input_size)
# Training Loop
mb = master_bar(range(nb_epochs))
for epoch in mb:
    loss = 0
    for iter_idx in progress_bar(range(nb_iterations), parent=mb):
        # Let's approximate the identity function
        expected = inputs[iter_idx]
        # TODO: forward the input
        outputs = None
        # TODO: backpropagate the loss gradient
        mlp.backward(None)
        # TODO: update the MLP parameters
        mlp.update_params(0)
        # TODO: update the total loss (do not divide by the number of terms)
        loss += 0
        mb.child.comment = f"running loss {loss / float(outputs.size * (iter_idx + 1))}"
    mb.first_bar.comment = f"completed epoch {epoch+1}"
    mb.write(f"Epoch {epoch+1}: MSE {loss / float(outputs.size * nb_iterations)}")

This time, you really deserve a treat!
With all your work, you are now able to implement fully connected layers, activation, and backpropagation for a full MLP network. In the same time, you gain (hopefully) better knowledge about the linear algebra and calculus hidden behind those with the computational graph and the implications of the chain rule.

Congratulations again, have a well-deserved rest!

*Spoiler alert: it's just the beginning of the journey as there are many other layers that compose common deep learning architectures and other optimizers. But from now on, you will only have to apply the same logic and think about how to implement the forward method and how to backpropagate through the operator gate*

![Most interesting man](https://i.imgflip.com/2p0lib.jpg "Most interesting man")
*Source: imgflip.com*